In [1]:
c = '00000001'
print(int(str(c), 2))

c = '11111111'
print(int(str(c), 2))

c = '1000'
print(int(str(c), 2))


# 1	0	0	0	1	1	0	0	0	1	1	1
minimo = '000100010001'
maximo = '100011111111'
print(int(str(minimo), 2))
print(int(str(maximo), 2))

c = '001000000010'
print(int(str(c), 2))

1
255
8
273
2303
514


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MaxNLocator
from learntools.time_series.utils import plot_periodogram, seasonal_plot
from learntools.time_series.style import *
import seaborn as sns
from IPython.display import Markdown, display
from pathlib import Path
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Function to display markdown
def printmd(string):
    display(Markdown(string))

# Define directory and read data
comp_dir = Path('../input/amazon-product-reviews')
electronics_data = pd.read_csv(
    comp_dir / "ratings_Electronics (1).csv",
    dtype={'rating': 'int8'},
    names=['userId', 'productId', 'rating', 'timestamp'],
    index_col=None,
    header=0
)

# Display some basic information
printmd(f"Number of Rating: {electronics_data.shape[0]:,}")
printmd(f"Columns: {np.array2string(electronics_data.columns.values)}")
printmd(f"Number of Users: {len(electronics_data.userId.unique()):,}")
printmd(f"Number of Products: {len(electronics_data.productId.unique()):,}")
electronics_data.describe()['rating'].reset_index()

# Check for missing values
printmd('**Number of missing values**:')
pd.DataFrame(
    electronics_data.isnull().sum().reset_index()
).rename(columns={0: "Total missing", "index": "Columns"})

# Process data by date
data_by_date = electronics_data.copy()
data_by_date.timestamp = pd.to_datetime(electronics_data.timestamp, unit="s")
data_by_date = data_by_date.sort_values(by="timestamp", ascending=False).reset_index(drop=True)

printmd("Number of Ratings each day:")
data_by_date.groupby("timestamp")["rating"].count().tail(10).reset_index()

# Add year and month columns
data_by_date["year"] = data_by_date.timestamp.dt.year
data_by_date["month"] = data_by_date.timestamp.dt.month
rating_by_year = data_by_date.groupby(["year", "month"])["rating"].count().reset_index()

# Create date column and plot data
rating_by_year["date"] = pd.to_datetime(rating_by_year["year"].astype(str) + "-" + rating_by_year["month"].astype(str) + "-1")
rating_by_year.plot(x="date", y="rating")
plt.title("Number of Ratings over Years")
plt.show()

# Group by product and calculate statistics
rating_by_product = electronics_data.groupby("productId").agg({
    "userId": "count",
    "rating": "mean"
}).rename(columns={"userId": "Number of Ratings", "rating": "Average Rating"}).reset_index()

# Filter top-rated products
cutoff = 50
top_rated = rating_by_product.loc[
    rating_by_product["Number of Ratings"] > cutoff
].sort_values(by="Average Rating", ascending=False).reset_index(drop=True)

# Define TensorFlow model
class RankingModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        # Embedding layers for users and products
        self.user_embeddings = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=unique_userIds, mask_token=None),
            tf.keras.layers.Embedding(len(unique_userIds) + 1, embedding_dimension)
        ])

        self.product_embeddings = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=unique_productIds, mask_token=None),
            tf.keras.layers.Embedding(len(unique_productIds) + 1, embedding_dimension)
        ])

        # Ratings layers
        self.ratings = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(1)
        ])

    def call(self, userId, productId):
        user_embeddings = self.user_embeddings(userId)
        product_embeddings = self.product_embeddings(productId)
        return self.ratings(tf.concat([user_embeddings, product_embeddings], axis=1))

class amazonModel(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        self.ranking_model = RankingModel()
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def compute_loss(self, features, training=False):
        rating_predictions = self.ranking_model(features["userId"], features["productId"])
        return self.task(labels=features["rating"], predictions=rating_predictions)

# Filter recent data
cutoff_no_rat = 50
cutoff_year = 2011
recent_data = data_by_date.loc[data_by_date["year"] > cutoff_year]
print(f"Number of Rating: {recent_data.shape[0]:,}")
print(f"Number of Users: {len(recent_data.userId.unique()):,}")
print(f"Number of Products: {len(recent_data.productId.unique()):,}")
del data_by_date  # Free up memory

recent_prod = recent_data.loc[
    recent_data.groupby("productId")["rating"].transform('count').ge(cutoff_no_rat)
].reset_index(drop=True).drop(["timestamp", "year", "month"], axis=1)
del recent_data  # Free up memory

# Prepare data for training
userIds = recent_prod.userId.unique()
productIds = recent_prod.productId.unique()
total_ratings = len(recent_prod.index)

ratings = tf.data.Dataset.from_tensor_slices({
    "userId": tf.cast(recent_prod.userId.values, tf.string),
    "productId": tf.cast(recent_prod.productId.values, tf.string),
    "rating": tf.cast(recent_prod.rating.values, tf.int8)
})

# Shuffle and split data
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
train = shuffled.take(int(total_ratings * 0.8))
test = shuffled.skip(int(total_ratings * 0.8)).take(int(total_ratings * 0.2))

unique_productIds = productIds
unique_userIds = userIds

# Compile and train model
model = amazonModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()
model.fit(cached_train, epochs=10)

# Evaluate model
model.evaluate(cached_test, return_dict=True)

# Recommend products for a random user
user_rand = userIds[123]
test_rating = {}
for m in test.take(5):
    test_rating[m["productId"].numpy()] = RankingModel()(
        tf.convert_to_tensor([user_rand]), tf.convert_to_tensor([m["productId"]])
    )

print(f"Top 5 recommended products for User {user_rand}:")
for m in sorted(test_rating, key=test_rating.get, reverse=True):
    print(m.decode())





Los sistemas de calificación ponderada se utilizan para puntuar la calificación de cada película. Esta es la fórmula de la puntuación ponderada.
WR = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C
R es la puntuación media del artículo.
v es el número de votos del artículo.
m es el mínimo de votos necesarios para figurar en los artículos populares (definido por > percentil 80 del total de votos).
C es la valoración media de todo el conjunto de datos.



Vemos que cada método tiene su punto fuerte. Lo mejor sería poder combinar todos esos puntos fuertes y ofrecer una recomendación mejor. Esta idea nos lleva a otra mejora de la recomendación, que es el método híbrido. Por ejemplo, podemos combinar las recomendaciones de filtrado colaborativo basadas en el contenido y en los elementos para aprovechar las características de ambos dominios (géneros e interacción usuario-elemento).

Traducción realizada con la versión gratuita del traductor DeepL.com

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.



electronics_data=pd.read_csv("/kaggle/input/amazon-product-reviews/ratings_Electronics (1).csv",names=['userId', 'productId','Rating','timestamp'])


# Display the data

electronics_data.head()

#Shape of the data
electronics_data.shape

#Taking subset of the dataset
electronics_data=electronics_data.iloc[:1048576,0:]

#Check the datatypes
electronics_data.dtypes


electronics_data.info()



#Five point summary 

electronics_data.describe()['Rating'].T


#Find the minimum and maximum ratings
print('Minimum rating is: %d' %(electronics_data.Rating.min()))
print('Maximum rating is: %d' %(electronics_data.Rating.max()))

#Check for missing values
print('Number of missing values across columns: \n',electronics_data.isnull().sum())


# Check the distribution of the rating
with sns.axes_style('white'):
    g = sns.factorplot("Rating", data=electronics_data, aspect=2.0,kind='count')
    g.set_ylabels("Total number of ratings")


print("Total data ")
print("-"*50)
print("\nTotal no of ratings :",electronics_data.shape[0])
print("Total No of Users   :", len(np.unique(electronics_data.userId)))
print("Total No of products  :", len(np.unique(electronics_data.productId)))


#Dropping the Timestamp column

electronics_data.drop(['timestamp'], axis=1,inplace=True)

#Analysis of rating given by the user 

no_of_rated_products_per_user = electronics_data.groupby(by='userId')['Rating'].count().sort_values(ascending=False)

no_of_rated_products_per_user.head()

no_of_rated_products_per_user.describe()

quantiles = no_of_rated_products_per_user.quantile(np.arange(0,1.01,0.01), interpolation='higher')

plt.figure(figsize=(10,10))
plt.title("Quantiles and their Values")
quantiles.plot()
# quantiles with 0.05 difference
plt.scatter(x=quantiles.index[::5], y=quantiles.values[::5], c='orange', label="quantiles with 0.05 intervals")
# quantiles with 0.25 difference
plt.scatter(x=quantiles.index[::25], y=quantiles.values[::25], c='m', label = "quantiles with 0.25 intervals")
plt.ylabel('No of ratings by user')
plt.xlabel('Value at the quantile')
plt.legend(loc='best')
plt.show()

print('\n No of rated product more than 50 per user : {}\n'.format(sum(no_of_rated_products_per_user >= 50)) )

#Getting the new dataframe which contains users who has given 50 or more ratings

new_df=electronics_data.groupby("productId").filter(lambda x:x['Rating'].count() >=50)

no_of_ratings_per_product = new_df.groupby(by='productId')['Rating'].count().sort_values(ascending=False)

fig = plt.figure(figsize=plt.figaspect(.5))
ax = plt.gca()
plt.plot(no_of_ratings_per_product.values)
plt.title('# RATINGS per Product')
plt.xlabel('Product')
plt.ylabel('No of ratings per product')
ax.set_xticklabels([])

plt.show()


#Average rating of the product 

new_df.groupby('productId')['Rating'].mean().head()


new_df.groupby('productId')['Rating'].mean().sort_values(ascending=False).head()

#Total no of rating for product

new_df.groupby('productId')['Rating'].count().sort_values(ascending=False).head()

ratings_mean_count = pd.DataFrame(new_df.groupby('productId')['Rating'].mean())

ratings_mean_count['rating_counts'] = pd.DataFrame(new_df.groupby('productId')['Rating'].count())

ratings_mean_count.head()
ratings_mean_count['rating_counts'].max()
plt.figure(figsize=(8,6))
plt.rcParams['patch.force_edgecolor'] = True
ratings_mean_count['rating_counts'].hist(bins=50)

plt.figure(figsize=(8,6))
plt.rcParams['patch.force_edgecolor'] = True
ratings_mean_count['Rating'].hist(bins=50)


plt.figure(figsize=(8,6))
plt.rcParams['patch.force_edgecolor'] = True
sns.jointplot(x='Rating', y='rating_counts', data=ratings_mean_count, alpha=0.4)

popular_products = pd.DataFrame(new_df.groupby('productId')['Rating'].count())
most_popular = popular_products.sort_values('Rating', ascending=False)
most_popular.head(30).plot(kind = "bar")



from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split


#Reading the dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(new_df,reader)


#Splitting the dataset
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)


# Use user_based true/false to switch between user-based or item-based collaborative filtering
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)



# run the trained model against the testset
test_pred = algo.test(testset)


# get RMSE
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)






new_df1=new_df.head(10000)
ratings_matrix = new_df1.pivot_table(values='Rating', index='userId', columns='productId', fill_value=0)
ratings_matrix.head()
ratings_matrix.shape


X = ratings_matrix.T
X.head()

X.shape
X1 = X

#Decomposing the Matrix
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape


#Correlation Matrix

correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape


X.index[75]

i = "B00000K135"

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID


correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

Recommend = list(X.index[correlation_product_ID > 0.65])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:24]









In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

# Split
from sklearn.model_selection import train_test_split

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv("/kaggle/input/amazon-product-reviews/ratings_Electronics (1).csv",
                             names=['userId', 'productId','rating','timestamp'])

df.head()

print("Total Reviews:",df.shape[0])
print("Total Columns:",df.shape[1])

# Taking subset of the dataset
df = df.iloc[:5000,0:]

print("Total Reviews:",df.shape[0])
print("Total Columns:",df.shape[1])

print("Total number of ratings :",df.rating.nunique())
print("Total number of users   :", df.userId.nunique())
print("Total number of products  :", df.productId.nunique())

df.info()

# Check missing value
df.isnull().sum()

# Check Duplicate data
df[df.duplicated()].any()

# rating describe summary 
df.describe()['rating']

print("Unique value of Rating:",df.rating.unique())

# Find the minimum and maximum ratings
print('Minimum rating is: %d' %(df.rating.min()))
print('Maximum rating is: %d' %(df.rating.max()))

# Average rating of products
ratings = pd.DataFrame(df.groupby('productId')['rating'].mean())
ratings['ratings_count'] = pd.DataFrame(df.groupby('productId')['rating'].count())
ratings['ratings_average'] = pd.DataFrame(df.groupby('productId')['rating'].mean())
ratings.head(10)

plt.figure(figsize=(10,4))
ratings['rating'].hist(bins=70)

sns.jointplot(x='rating',y='ratings_count',data=ratings,alpha=0.5)

# Most top 30 products
popular_products = pd.DataFrame(df.groupby('productId')['rating'].count())
most_popular = popular_products.sort_values('rating', ascending=False)
most_popular.head(30).plot(kind = "bar",figsize=(12, 4))

vote_counts = ratings[ratings['ratings_count'].notnull()]['ratings_count'].astype('int')
vote_averages = ratings[ratings['ratings_average'].notnull()]['ratings_average'].astype('int')
C = vote_averages.mean()
print("Average rating of product across the whole dataset is",C)

m = vote_counts.quantile(0.95)
print("Minimum votes required to be listed in the chart is",m)

ratings.head()

qualified = ratings[(ratings['ratings_count'] >= m) & (ratings['ratings_count'].notnull()) & (ratings['ratings_average'].notnull())][['ratings_count', 'ratings_average']]

qualified['ratings_count'] = qualified['ratings_count'].astype('int')
qualified['ratings_average'] = qualified['ratings_average'].astype('int')
qualified.head().sort_values(by='ratings_count', ascending=False)

qualified.shape

def weighted_rating(x):
    v = x['ratings_count']
    R = x['ratings_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

qualified['wr'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('wr', ascending=False).head(20)
qualified.head(10)

# Add color
from matplotlib import cm
color = cm.inferno_r(np.linspace(.4, .8, 30))

rating_plot_count = qualified['ratings_count'].plot.bar(figsize=(12, 4),color=color)
rating_plot_count.set_title("Rating Count Bar-Plot")
rating_plot_count.set_xlabel("productId")
rating_plot_count.set_ylabel("Count")


rating_plot_avg = qualified['ratings_average'].plot.bar(figsize=(12, 4),color=color)
rating_plot_avg.set_title("Rating Average Bar-Plot")
rating_plot_avg.set_xlabel("productId")
rating_plot_avg.set_ylabel("rating")

wr_plot = qualified['wr'].plot.bar(figsize=(12, 4),color=color)
wr_plot.set_title("Weight Rating Bar-Plot")
wr_plot.set_xlabel("productId")
wr_plot.set_ylabel("rating")





reader = Reader()
df.head()

data = Dataset.load_from_df(df[['userId', 'productId', 'rating']], reader)

# Use the famous SVD algorithm
svd = SVD()

# Run 5-fold cross-validation and then print results
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()
svd.fit(trainset)

df[df['userId'] == 'AKM1MP6P0OYPR']

svd.predict(uid='A17HMM1M7T9PJ1', iid='0970407998', r_ui=None)
svd.predict(uid='A17HMM1M7T9PJ1', iid='0970407998', r_ui=None).est


df_users=df.groupby('userId').filter(lambda x: x['rating'].count()>=50)
df_users.head()
df_users.shape


matrix=pd.pivot_table(data=df_users, values='rating', index='userId',columns='productId')
matrix.head()

# Function that takes in productId and useId as input and outputs up to 5 most similar products.
def hybrid_recommendations(userId, productId):
    
    # Get the Id of the top five products that are correlated with the ProductId chosen by the user.
    top_five=matrix.corrwith(matrix[productId]).sort_values(ascending=False).head(5)
    
    # Predict the ratings the user might give to these top 5 most correlated products.
    est_rating=[]
    for x in list(top_five.index):
        if str(top_five[x])!='nan':
            est_rating.append(svd.predict(userId, iid=x, r_ui=None).est)
           
    return pd.DataFrame({'productId':list(top_five.index)[:len(est_rating)], 'estimated_rating':est_rating}).sort_values(by='estimated_rating', ascending=False).reset_index(drop=True)


hybrid_recommendations('A2NYK9KWFMJV4Y', 'B00LI4ZZO8')

# df.head()

# df['userId'].value_counts()

# # Check specific userId review
# df[df['userId'] == 'A3LDPF5FMB782Z']

# # predict based on this data
# svd.predict('A3LDPF5FMB782Z', '140053271X', 5.0)





In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
# Build a model.
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        self.user_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=unique_userIds, mask_token=None),
                                        # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(unique_userIds)+1, embedding_dimension)
                                    ])

        self.product_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=unique_productIds, mask_token=None),
                                    # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(unique_productIds)+1, embedding_dimension)
                                    ])
        # Set up a retrieval task and evaluation metrics over the
        # entire dataset of candidates.
        self.ratings = tf.keras.Sequential([
                            tf.keras.layers.Dense(256, activation="relu"),
                            tf.keras.layers.Dense(64,  activation="relu"),
                            tf.keras.layers.Dense(1)
                              ])
    def call(self, userId, productId):
        user_embeddings  = self.user_embeddings (userId)
        product_embeddings = self.product_embeddings(productId)
        return self.ratings(tf.concat([user_embeddings,product_embeddings], axis=1))

# Build a model.
class amazonModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer   = tfrs.tasks.Ranking(
                                                    loss    =  tf.keras.losses.MeanSquaredError(),
                                                    metrics = [tf.keras.metrics.RootMeanSquaredError()])
            

    def compute_loss(self, features, training=False):
        rating_predictions = self.ranking_model(features["userId"], features["productId"]  )

        return self.task( labels=features["rating"], predictions=rating_predictions)
    





cutoff_no_rat = 50    ## Only count products which received more than or equal 50
cutoff_year   = 2011  ## Only count Rating after 2011
recent_data   = data_by_date.loc[data_by_date["year"] > cutoff_year]
print("Number of Rating: {:,}".format(recent_data.shape[0]) )
print("Number of Users: {:,}".format(len(recent_data.userId.unique()) ) )
print("Number of Products: {:,}".format(len(recent_data.productId.unique())  ) )
del data_by_date  ### Free up memory ###
recent_prod   = recent_data.loc[recent_data.groupby("productId")["rating"].transform('count').ge(cutoff_no_rat)].reset_index(
                    drop=True).drop(["timestamp","year","month"],axis=1)
del recent_data  ### Free up memory ###


userIds    = recent_prod.userId.unique()
productIds = recent_prod.productId.unique()
total_ratings= len(recent_prod.index)


ratings = tf.data.Dataset.from_tensor_slices( {"userId":tf.cast( recent_prod.userId.values  ,tf.string),
                                "productId":tf.cast( recent_prod.productId.values,tf.string),
                                "rating":tf.cast( recent_prod.rating.values  ,tf.int8,) } )


tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take( int(total_ratings*0.8) )
test = shuffled.skip(int(total_ratings*0.8)).take(int(total_ratings*0.2))

unique_productIds = productIds
unique_userIds    = userIds


model = amazonModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad( learning_rate=0.1 ))
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()
model.fit(cached_train, epochs=10)


# Evaluate.
model.evaluate(cached_test, return_dict=True)


user_rand = userIds[123]
test_rating = {}
for m in test.take(5):
    test_rating[m["productId"].numpy()]=RankingModel()(tf.convert_to_tensor([user_rand]),tf.convert_to_tensor([m["productId"]]))



print("Top 5 recommended products for User {}: ".format(user_rand))
for m in sorted(test_rating, key=test_rating.get, reverse=True):
    print(m.decode())






In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate

sns.set_style("ticks")

# Import both datasets

df5=pd.read_csv('../input/amazon-cell-phones-reviews/20191226-items.csv')
df6=pd.read_csv('../input/amazon-cell-phones-reviews/20191226-reviews.csv')

df5.rename(columns={'rating':'avgRating'}, inplace=True)
columns = ['url', 'reviewUrl', 'totalReviews', 'originalPrice']
df5.drop(columns, inplace=True, axis=1)
# Drop uneeded columns before merging both datasets

columns = ['date', 'verified', 'title', 'body', 'helpfulVotes']
df6.drop(columns, inplace=True, axis=1)

df6
# Merging 2 df to create training data for the SVD

ratings = pd.merge(df5, df6, how='inner', on='asin')

columns = ['brand', 'price', 'image', 'avgRating']
ratings.drop(columns, inplace=True, axis=1)

ratings = ratings[['name', 'asin', 'title', 'rating']]
ratings = ratings.sort_values(by=['name'], ascending=True)
ratings = ratings.reset_index(drop=True)

ratings


# Create a pivot table to see how the data columns correspond to one another

matrix=pd.pivot_table(data=ratings[['name', 'asin', 'rating']], values='rating', index='name',columns='asin')
matrix.head()

# Initialize the SVD model and train the model on the created dataset

svd = SVD()
reader = Reader()
data = Dataset.load_from_df(ratings[['name', 'asin', 'rating']], reader)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()
svd.fit(trainset)

# Function that takes in productId and userId as input and outputs up to 5 most similar products.
def hybrid_recommendations(userId, productId):
    
    # Get the Id of the top five products that are correlated with the ProductId chosen by the user.
    top_five=matrix.corrwith(matrix[productId]).sort_values(ascending=False).head(15)
    
    # Predict the ratings the user might give to these top 5 most correlated products.
    est_rating=[]
    for x in list(top_five.index):
        if str(top_five[x])!='nan':
            est_rating.append(svd.predict(userId, iid=x, r_ui=None).est)
           
    return pd.DataFrame({'productId':list(top_five.index)[:len(est_rating)], 'estimated_rating':est_rating}).sort_values(by='estimated_rating', ascending=False).reset_index(drop=True)


find_product = hybrid_recommendations('John', 'B018OMP8ES')

# Function that maps the productId to the productName
def product_mapping(recommender_df): 
    rows_list = []
    rows_list2 = []

    # Append productName into finalDf
    for x in recommender_df['productId']:
        cut_row = ratings.loc[ratings['asin'] == x]
        cut_row = cut_row['title'][0:1].values
        rows_list.append(cut_row[0])

    # Copy over the estimated ratings into the finalDf
    for i in recommender_df['estimated_rating']:
        rows_list2.append(i)

    # Creating the finalDf
    cut_dict = {"product": rows_list, "estimated_rating": rows_list2}
    cut_df = pd.DataFrame(cut_dict)
    
    return cut_df

# Test 1
product_mapping(find_product)

# Test 2
product_mapping(hybrid_recommendations('Peter ', 'B077T4MVZ6'))

# Test 3
product_mapping(hybrid_recommendations('Sarah ', 'B081H6STQQ'))



# Save the model into a joblib file
from joblib import dump, load

dump(svd, 'svd.joblib')









In [ ]:
# To compute the accuracy of models
from surprise import accuracy

# Class is used to parse a file containing ratings, data should be in structure - user ; item ; rating
from surprise.reader import Reader

# Class for loading datasets
from surprise.dataset import Dataset

# For tuning model hyperparameters
from surprise.model_selection import GridSearchCV

# For splitting the rating data in train and test datasets
from surprise.model_selection import train_test_split

# For implementing similarity-based recommendation system
from surprise.prediction_algorithms.knns import KNNBasic

# For implementing matrix factorization based recommendation system
from surprise.prediction_algorithms.matrix_factorization import SVD

# for implementing K-Fold cross-validation
from surprise.model_selection import KFold

# For implementing clustering-based recommendation system
from surprise import CoClustering


def precision_recall_at_k(model, k = 10, threshold = 3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user
    user_est_true = defaultdict(list)

    # Making predictions on the test data
    predictions = model.test(testset)

    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key = lambda x: x[0], reverse = True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. Therefore, we are setting Precision to 0 when n_rec_k is 0

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. Therefore, we are setting Recall to 0 when n_rel is 0

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    # Mean of all the predicted precisions are calculated.
    precision = round((sum(prec for prec in precisions.values()) / len(precisions)), 3)

    # Mean of all the predicted recalls are calculated.
    recall = round((sum(rec for rec in recalls.values()) / len(recalls)), 3)

    accuracy.rmse(predictions)

    print('Precision: ', precision) # Command to print the overall precision

    print('Recall: ', recall) # Command to print the overall recall

    print('F_1 score: ', round((2*precision*recall)/(precision+recall), 3)) # Formula to compute the F-1 score



# Instantiating Reader scale with expected rating scale
reader = Reader(rating_scale=(1, 5))

# Loading the rating dataset
df = Dataset.load_from_df(df[['user_id', 'prod_id', 'rating']], reader)

# Splitting the data into train and test datasets
trainset, testset = train_test_split(df, test_size=0.7, random_state=42)



# Declaring the similarity options
sim_options = {'name': 'cosine', 'user_based': True}

# Initialize the KNNBasic model using sim_options declared, Verbose = False, and setting random_state = 1
algo_knn_user = KNNBasic(sim_options=sim_options, verbose=False, random_state=1)

# Fit the model on the training data
algo_knn_user.fit(trainset)

# Let us compute precision@k, recall@k, and f_1 score using the precision_recall_at_k function defined above
precision_recall_at_k(algo_knn_user)


# Predicting rating for a sample user with an interacted product
algo_knn_user.predict('A3LDPF5FMB782Z', '1400501466', r_ui=5, verbose=True)


# Unique user_id where prod_id is not equal to "1400501466"
df_final.loc[df_final['prod_id'] != "1400501466", 'user_id'].unique()


# Predicting rating for a sample user with a non interacted product
algo_knn_user.predict('A34BZM6S9L7QI4', '1400501466', verbose=True)

# Setting up parameter grid to tune the hyperparameters
param_grid = {'k': [20, 30, 40], 'min_k': [3, 6, 9],
              'sim_options': {'name': ['msd', 'cosine', 'pearson'],
                              'user_based': [True]}
              }

# Performing 3-fold cross-validation to tune the hyperparameters
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=3, n_jobs=-1)

# Fitting the data
gs.fit(df)

# Best RMSE score
print(gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])



# Using the optimal similarity measure for user-user based collaborative filtering
sim_options = {'name': 'cosine',
               'user_based': True}

# Creating an instance of KNNBasic with optimal hyperparameter values
similarity_algo_optimized = KNNBasic(sim_options=sim_options, k=40, min_k=6, verbose=False, random_state=1)

# Training the algorithm on the trainset
similarity_algo_optimized.fit(trainset)

# Let us compute precision@k and recall@k also with k =10
precision_recall_at_k(similarity_algo_optimized)

# sim_user_user_optimized model to recommend for userId "A3LDPF5FMB782Z" and productId 1400501466
similarity_algo_optimized.predict('A3LDPF5FMB782Z', '1400501466', r_ui=5, verbose=True)


# sim_user_user_optimized model to recommend for userId "A34BZM6S9L7QI4" and productId "1400501466"
similarity_algo_optimized.predict('A34BZM6S9L7QI4', '1400501466', verbose=True)


# 0 is the inner id of the above user
similarity_algo_optimized.get_neighbors(0, k=5)


def get_recommendations(data, user_id, top_n, algo):

    # Creating an empty list to store the recommended product ids
    recommendations = []

    # Creating an user item interactions matrix
    user_item_interactions_matrix = data.pivot(index = 'user_id', columns = 'prod_id', values = 'rating')

    # Extracting those product ids which the user_id has not interacted yet
    non_interacted_products = user_item_interactions_matrix.loc[user_id][user_item_interactions_matrix.loc[user_id].isnull()].index.tolist()

    # Looping through each of the product ids which user_id has not interacted yet
    for item_id in non_interacted_products:

        # Predicting the ratings for those non interacted product ids by this user
        est = algo.predict(user_id, item_id).est

        # Appending the predicted ratings
        recommendations.append((item_id, est))

    # Sorting the predicted ratings in descending order
    recommendations.sort(key = lambda x: x[1], reverse = True)

    return recommendations[:top_n] # Returing top n highest predicted rating products for this user


# Making top 5 recommendations for user_id "A3LDPF5FMB782Z" with a similarity-based recommendation engine
recommendations = get_recommendations(df_final, 'A3LDPF5FMB782Z', 5, algo_knn_user)


# Building the dataframe for above recommendations with columns "prod_id" and "predicted_ratings"
pd.DataFrame(recommendations, columns=['prod_Id', 'predicted_ratings'])


